<a href="https://colab.research.google.com/github/dylanscoggins/CSCE4205Project/blob/dylan/Group0Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Imports

In [14]:
import pandas as pd
import os
import re

# Mount Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Helper Functions

In [15]:
# credit to https://www.kaggle.com/reddieeddie
def get_row(df, col, name):
  return df[df[col]==name]

# Data Sets

In [ ]:
# download .csv data sets from GitHub repository
# link to repo: https://github.com/dylanscoggins/CSCE4205Project
pokemonDataSet01 = 'https://raw.githubusercontent.com/dylanscoggins/CSCE4205Project/main/PokemonDataSet01.csv'
ds1 = pd.read_csv(pokemonDataSet01)

pokemonDataSet02 = 'https://raw.githubusercontent.com/dylanscoggins/CSCE4205Project/main/PokemonDataSet02.csv'
ds2 = pd.read_csv(pokemonDataSet02)

# associate images with pokemon in ds2
# credit to https://www.kaggle.com/reddieeddie for the framework to set up Pokédex
pokedex = '/content/gdrive/MyDrive/CSCE 4205/Project Data/Images/'
fileNames = os.listdir(pokedex)
pokeNames, pokeTypes = [], []
for path in os.listdir(pokedex):
  name = re.search(r"^([^.]*).*", path).group(1)
  pokeNames.append(name.capitalize())
  pokeTypes.append(ds2['Type1'][int(get_row(ds2, 'Name', name).index.values)])

numImgs = len(filenames)
# print("Number of pokemon: ", numImgs)
print("Pokédex contains", numImgs, "Pokémon")

col1 = fileNames
col2 = pokeNames
col3 = pokeTypes

pokemonData = pd.DataFrame({'Image':col1, 'Pokémon':col2, 'Primary Type':col3})
pokemonData